In [52]:
import astropy.io.fits as pf
import numpy as np
import matplotlib.pyplot as plt
import math
import os

### Directorios del proyecto

In [47]:
DIR_PROYECTO  = '/home/matias/tesis/'
DIR_MOSAICOS  = DIR_PROYECTO + 'fits_face/'
DIR_CATÁLOGOS = DIR_PROYECTO + 'catalogues/'
DIR_SEGMENTACIONES = DIR_PROYECTO + 'segmentation/'
DIR_MORFOLOGÍAS    = DIR_PROYECTO + 'morfologías/'
DIR_DATOS_EAGLE    = DIR_PROYECTO + 'tablas/'
DIR_LOTZ = DIR_PROYECTO + 'gm20/'

### Definición de valores de filtro

In [48]:
U = 121671348.887
G = 168273491.446
R = 178848396.083
I = 69397708.7252
Z = 59157262.0017
U_PRI = 126207773.267
G_PRI = 149732062.76
R_PRI = 188318397.068
I_PRI = 67579475.1401
Z_PRI = 50496673.9991

### Carga del SQL query a la base de datos de EAGLE

In [7]:
import pandas as pd

encabezado = [
  'id',
  'sfr',
  'm_dm',
  'm_gs',
  'm_st',
  'r_hm',
  'u_mag',
  'g_mag',
  'r_mag'
]

columnas = [0,1,2,3,4,5,10,11,12]

eagle_data = pd.read_csv(DIR_EAGLE_DATA + 'eagle_db_fits_27.dat', names=encabezado, usecols=columnas)

eagle_data['color_gr'] = eagle_data['g_mag'] - eagle_data['r_mag']

# Magnitudes del SDSS a U-B: U-B = 0.78 * (u - g) - 0.88
eagle_data['color_ub'] = 0.78 * (eagle_data['u_mag'] - eagle_data['r_mag']) - 0.88

### Generación del mapa de pesos

In [ ]:
# Primero creo un HDU object
hdu = fits.PrimaryHDU()

# Lo lleno con random
#hdu.data = np.random.random((256,256))
# Lo lleno con 1
hdu.data = np.ones((256, 256))

# Obtiene la cabecera primaria
#cabecera = hdu[0].header

hdu.writeto('wht1.fits', clobber=True)

### Generación de archivos para correr el programa de Lotz en modo 'batch'
`zeropt.dat` tiene las magnitudes de base de los mosaicos y `run_gmorph.pro` los mosaicos a procesar. Se generan en el mismo bucle para que el orden de las magnitudes se corresponda con el de los mosaicos.

In [51]:
with open('zeropt.dat', 'wt') as zeropt, open('run_gmorph.pro', 'wt') as run_gmorph:

    for basename_mosaico in os.listdir(DIR_MOSAICOS):

        with pf.open(DIR_MOSAICOS + basename_mosaico) as fits_mosaico:
            flujo_total_filtro    = np.sum(fits_mosaico[0].data) / R # flujo total / valor de filtro
            magnitud_total_filtro = -2.5 * np.log10(flujo_total_filtro / (3.631e6 * 4.8532896**2))

            zeropt.write(str(magnitud_total_filtro) + '\n')


        nombre = os.path.splitext(basename_mosaico)[0] # separa el nombre del archivo de su extensión

        comando = "get_gmorph_new, 0, '%s', '%s', '%s', '%s', '%s'" % ( \
            # catálogo
            DIR_CATÁLOGOS      + nombre + '.cat',      \
            # mosaico
            DIR_MOSAICOS       + nombre + '.fits',     \
            # mapa de pesos
            DIR_PROYECTO       + 'wht1' + '.fits',     \
            # mapa de segmentación
            DIR_SEGMENTACIONES + nombre + '.seg.fits', \
            # archivo de salida
            DIR_MORFOLOGÍAS    + nombre + '.morph'     \
        )

        # Cada línea luce como: get_gmorph_new, 0, 'xxx.cat', 'xxx.fits', 'wht1.fits', 'xxx.fits', 'xxx.morph'
        # "where 0 is the line number in xxx.cat of the starting object"
        run_gmorph.write(comando + '\n')

    run_gmorph.write('exit')

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.io import ascii 
import csv
import sys
import h5py


gini_eag = np.zeros(dim)
m20_eag = np.zeros(dim)
asi_eag = np.zeros(dim)
s_eag = np.zeros(dim)
i=0
#with open('out_face_r_test.morph', 'r') as Gf:
with open('out_face_r.morph', 'r') as Gf:    
    for line in Gf:
        list_eag = line.split()
        gini_eag[i] = list_eag[19]
        m20_eag[i] = list_eag[20]
        i = i +1
Gf.close()

print(gini_eag)

F = 5 * (-0.7783 - 0.14 * m20_eag + gini_eag)/1.00975

# Illustris
# cam0:10654 cam1:10618 cam2:10639 cam3:10620
gini_ill = np.zeros(10654)
m20_ill = np.zeros(10654)
f = h5py.File('nonparametric_morphologies.hdf5','r') # Lee el archivo
gini_ill = f['Snapshot_135']['gSDSS']['Gini_cam0'][:] 
m20_ill = f['Snapshot_135']['gSDSS']['M20_cam0'][:] 

# Observations
#gini_obs1 = np.zeros(160)
#m20_obs1 = np.zeros(160)
#i=0
#with open('Gini_M20_obs1.dat', 'r') as Gf1:
#    for line in Gf1:
#        mylist = line.split(",")
#        gini_obs1[i] = float(mylist[0])
#        m20_obs1[i] = float(mylist[1])
#        i = i + 1
     
#Gini vs M20
plt.scatter(m20_ill,gini_ill, marker ='.', color='red')    
plt.scatter(m20_eag,gini_eag, marker ='.', color='green')
#plt.scatter(m20_obs1,gini_obs1, marker ='.', color='red') 
#plt.plot(m20_obs1,0.7783 + 0.14 * m20_obs1, '-', color='black')
plt.plot(m20_eag,0.7783 + 0.14 * m20_eag, '-', color='grey')
#d = 5* abs(-0.14*m20_obs1 + gini_obs1 - 0.778)/1.009
plt.title('Face/Random,r/g_band,z=0.1/0')
plt.xlabel('M20') 
plt.ylabel('Gini') 
plt.xlim(-0.5, -3)
plt.ylim(0.3, 0.75)
#plt.loglog()
plt.show() 

#Scatter plot coloreado por propiedad
x = m20_eag
y = gini_eag
#c = np.log10(m_st)
c = color_UB
print(max(c),min(c))

#index = np.where(c >= 5.0e13)
#print len(c)
#x = x[index]
#y = y[index]
#c = c[index]
#print len(c)

plt.scatter(x, y, s=10, c=c, alpha=0.5)
plt.xlim(-0.5, -3)
plt.ylim(0.3, 0.75)
plt.show

fig, ax = plt.subplots()
im = ax.scatter(x, y, s=10, c=c, cmap=plt.cm.jet)
#im = ax.scatter(x, y, c=c, cmap=plt.cm.jet, marker='.', s=0.1)

plt.xlim(-0.5, -3)
plt.ylim(0.3, 0.75)

#axes = plt.gca()
#axes.set_xlim([xmin,xmax])
#axes.set_ylim([ymin,ymax])

# Add a colorbar
fig.colorbar(im, ax=ax)




### Generación del mapa de pesos

In [ ]:

import numpy as np


#image = fits.getdata("niriH.fits")
image = fits.getdata('/home/spedrosa/Projects/Eagle/fits_illus/broadband_132699.fits')
#print image.shape

#full_fit_path = os.path.join(fits_path, fit)
    #print full_fit_path
    # read datacube
f = pf.open(image)
cube = f[15].data
header = f[15].header
f.close()

#I_mean = np.mean(image)

#print I_mean

#print(listsum(image))

#for i in image.shape

plt.imshow(image, cmap='gray')
#plt.show(image_data)
plt.colorbar()